In [16]:
import pandas as pd
import numpy as np
import os

In [17]:
path = 'Monthes/'

files = []
filesNoAdd = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.csv' in file:
            files.append(os.path.join(r, file))
            filesNoAdd.append(file)

for f in filesNoAdd:
    print(f)


01-2019.csv
02-2019.csv
03-2019.csv
04-2019.csv
05-2019.csv


In [18]:
monthresult = ['total sale of Goods', 'Status']
productDetail = ['Sku', 'UPC', 'Catalogue N', 'Title', 'Label' ,'Cost Price' ,'V.S.P.', 'Latest SOH']
header = productDetail.copy()

In [19]:
def findID(sku, Data):
    numb = 0
    for SKUref in Data['Sku']: 
        
        if str(sku) == str(SKUref): 
            return numb
        numb += 1
    return "This good with Sku of " + sku + " is not in data." 

In [20]:
def listOfGoods(files):
    listOfSku = []
    for file in files:
        rawData = pd.read_csv(file, encoding='latin1')
        for i in range(len(rawData['Sku'])):
                if rawData.loc[ i , 'Sku'] in listOfSku:
                       continue
                else:
                       listOfSku.append(rawData.loc[i,'Sku' ])
                       
    return listOfSku

In [21]:
def buildList(file, ide, productDetail, monthresult, rawData, sku,monthRawData, finalFile):

    for det in productDetail:
        if det =='Sku':
            continue
        monthRawData.loc[findID(sku, monthRawData),det] = rawData[det][ide]
    file = str(file).replace('.csv','')
    file = file.replace('Monthes/','')
    for head in monthresult:
        monthRawData.loc[findID(sku, monthRawData), file + ' ' + head] = rawData[head][ide]
#     if finalFile:
#         for  :
    return monthRawData

In [22]:
# buildList('01-2019.csv',0,productDetail, monthresult, rawData)

In [27]:
monthRawData

,Sku,UPC,Catalogue N,Title,Label,Cost Price,V.S.P.,Latest SOH,01-2019 total sale of Goods,01-2019 Status,02-2019 total sale of Goods,02-2019 Status,03-2019 total sale of Goods,03-2019 Status,04-2019 total sale of Goods,04-2019 Status,05-2019 total sale of Goods,05-2019 Status
0,734835,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,734836,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,734837,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,734838,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,734839,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,734840,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,734841,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,734843,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,734845,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,734848,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
monthRawData = pd.DataFrame(columns = list(header))

In [26]:
for file in filesNoAdd:
    monthRawData['Sku'] = listOfGoods(files)
    for head in monthresult:
        
        file = str(file).replace('.csv','')
        monthRawData[file + ' ' + head] = np.zeros(len(listOfGoods(files)))
monthRawData['Latest SOH'] = np.zeros(len(monthRawData['Sku']))

In [29]:
# a = np.zeros(shape=(1,len(header)))
# monthRawData = pd.DataFrame(columns = list(header))
fcount = len(files)
b = []
finalFile =False
for file in files:
    rawData = pd.read_csv(file , encoding='latin1')
    sid =0
    if file ==files[-1]:
        finalFile = False
#     b = 0
    for sku in rawData['Sku']:
        monthRawData = buildList(file, sid, productDetail, monthresult, rawData,sku,monthRawData, finalFile)
    #             Dataide = findID(sku, monthRawData)
    #             b = listAddition(Dataide, sid, fcount, productDetail, monthresult, rawData, monthRawData)
        sid +=1

In [11]:
 monthRawData.to_csv('allMonthes.csv',index=False)

In [13]:
totalGoodSale = []
count = 0
for sku in monthRawData['Sku']:
    saleOfGood = []
    for numB in monthresult:
        saleOfGood.append(monthRawData.iloc[count][numB])
    totalGoodSale.append(np.sum(saleOfGood))
    count += 1
monthRawData["total sale of Goods"] = totalGoodSale
status = []
for tot in totalGoodSale:
#     if tot == 0:
    if tot <= 0:
        status.append('Not moving')
    if (tot > 0) and (tot <=2):
        status.append('Slow moving')
    if tot > 2:
        status.append('Fast moving')
#
for j in range(len(filesNoAdd)):
    rawData = pd.read_csv(files[j])

    for i in range(len(status)):
#         print(i)
        try:
            if noStock(monthRawData.iloc[i]['Sku'],rawData):
                status[i] = 'Run out of stock'
        except:
            continue
#
monthRawData["Status"] = status



KeyError: 'total sale of Goods'